In [1]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawl(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
            
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 1000:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawl(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawl([start])

In [4]:
index = {i:k for i, k in enumerate(texts)}

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [6]:
!cat index.txt

0: https://m.habr.com/ru/news/t/494466/comments/
1: https://m.habr.com/ru/company/otus/blog/494480/
2: https://u.tmtm.ru/free
3: https://m.habr.com/ru/company/pt/blog/494486/
4: https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w
5: https://m.habr.com/ru/post/494520/
6: https://twitter.com/habr_com
7: https://m.habr.com/ru/post/494504/
8: https://habr.com/ru/all/?mobile=no
9: https://m.habr.com/ru/post/494478/
10: https://m.habr.com/ru/post/494514/
11: https://m.habr.com/ru/post/494460/
12: https://m.habr.com/ru/company/pt/blog/494472/
13: https://m.habr.com/ru/post/494488/
14: https://m.habr.com/ru/company/otus/blog/494476/
15: https://u.tmtm.ru/startup-barriers
16: https://m.habr.com/ru/company/wd/blog/493378/
17: https://zen.yandex.ru/habr
18: https://m.habr.com/ru/company/wd/blog/493378/comments/
19: https://m.habr.com/ru/post/494474/
20: https://habr.com/ru/post/481056#comments
21: https://habr.com/ru/company/habr/blog/494452/#comments
22: https://habr.com/ru/company/pt/blog/

# Задание №2

In [7]:
lemmatizer = Mystem()

In [8]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    text = text.translate(str.maketrans('', '', string.punctuation)).replace('—', '')
    text = re.sub(' +', ' ', text)
    lemmas = lemmatizer.lemmatize(text)
    texts_lemmatized[url] = ''.join(lemmas).strip()

# Задание №3

In [9]:
docs = [index[k] for k in index]

In [10]:
reverse_index = {}

for url in texts_lemmatized:
    for word in texts_lemmatized[url].split(' '):
        if not word in reverse_index:
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [11]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [12]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [13]:
query = f'{x1} | {x2} & !{x3}'
print(query)

отключать | глазго & !Institute


In [14]:
find(query)

{'https://habr.com/ru/all/?mobile=no',
 'https://habr.com/ru/company/top3dshop/blog/494294/',
 'https://habr.com/ru/company/top3dshop/blog/494294/#comments',
 'https://habr.com/ru/news/t/494414/?mobile=no',
 'https://habr.com/ru/post/492946/#comments',
 'https://habr.com/ru/post/494516/#habracut',
 'https://m.habr.com/ru/company/wd/blog/493378/',
 'https://m.habr.com/ru/company/wd/blog/493378/comments/',
 'https://twitter.com/habr_com',
 'https://www.youtube.com/channel/UCEgdi0XIXXZ-qJOFPf4JSKw',
 'https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w',
 'https://www.youtube.com/channel/UClgRkhTL3_hImCAmdLfDE4g'}

# Задание №4

In [15]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [16]:
pd.DataFrame(tfs).T

,https://m.habr.com/ru/news/t/494466/comments/,https://m.habr.com/ru/company/otus/blog/494480/,https://u.tmtm.ru/free,https://m.habr.com/ru/company/pt/blog/494486/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://m.habr.com/ru/post/494520/,https://twitter.com/habr_com,https://m.habr.com/ru/post/494504/,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/post/494478/,...,https://t.co/USfiL96xFA,https://t.co/TtBnpkMxFZ,http://NetAcad.com,https://m.habr.com/ru/company/odnoklassniki/blog/494260/,https://dev.twitter.com/overview/terms/policy,https://habr.com/ru/post/494520/#habracut,https://habr.com/ru/news/t/494500/#comments,https://www.3dprintingmedia.network/,https://habr.com/ru/company/pt/blog/494486/comments/?mobile=no,https://habr.com/ru/company/habr/blog/494082/#comments
минтруд,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
зарплата,0.000146,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
в,0.037345,0.028986,0.026702,0.021985,0.011747,0.02579,0.013519,0.024342,0.032237,0.019742,...,0.032778,0.026943,0.0,0.035177,0.0,0.024896,0.036561,0.0,0.022049,0.020107
нерабочий,0.000146,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
неделя,0.000438,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
стена…,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
физ,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000670
лестница,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
отжимание,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [17]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [18]:
pd.DataFrame(idfs,index=['']).T

,
минтруд,1.832581
зарплата,1.386294
в,0.162519
нерабочий,1.714798
неделя,0.400478
...,...
стена…,4.605170
физ,4.605170
лестница,4.605170
отжимание,4.605170


In [19]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [20]:
pd.DataFrame(tfidfs).T

,https://m.habr.com/ru/news/t/494466/comments/,https://m.habr.com/ru/company/otus/blog/494480/,https://u.tmtm.ru/free,https://m.habr.com/ru/company/pt/blog/494486/,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://m.habr.com/ru/post/494520/,https://twitter.com/habr_com,https://m.habr.com/ru/post/494504/,https://habr.com/ru/all/?mobile=no,https://m.habr.com/ru/post/494478/,...,https://t.co/USfiL96xFA,https://t.co/TtBnpkMxFZ,http://NetAcad.com,https://m.habr.com/ru/company/odnoklassniki/blog/494260/,https://dev.twitter.com/overview/terms/policy,https://habr.com/ru/post/494520/#habracut,https://habr.com/ru/news/t/494500/#comments,https://www.3dprintingmedia.network/,https://habr.com/ru/company/pt/blog/494486/comments/?mobile=no,https://habr.com/ru/company/habr/blog/494082/#comments
минтруд,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
зарплата,0.000202,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
в,0.006069,0.004711,0.00434,0.003573,0.001909,0.004191,0.002197,0.003956,0.005239,0.003208,...,0.005327,0.004379,0.0,0.005717,0.0,0.004046,0.005942,0.0,0.003583,0.003268
нерабочий,0.000250,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
неделя,0.000175,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
стена…,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
физ,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.003087
лестница,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
отжимание,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


# Задание №5

wip